In [1]:
import pybullet as p
import time
import numpy as np
import os
import sys
from collections import namedtuple

import _utils as u
import _utils_v2 as u2
from _utils_v2 import geom_shape, xyz_v2, s, CFG_Loader

In [2]:
u2.init_window()
path = './cfgs'
cfgs = CFG_Loader(path).get_result()

In [3]:
def get_data(cfgs, name):
    if name.startswith('R_'):
        name = name.replace('R_','')
        data = cfgs[name].RIGHT
        try:
            additional_name = data.motor_ops.stick_to
            if additional_name.startswith('R_'):
                additional_name = additional_name.replace('R_','')
        except:
            additional_name = None
    elif name.startswith('L_'):
        name = name.replace('L_','')
        data = cfgs[name].LEFT
        try:
            additional_name = data.motor_ops.stick_to
            if additional_name.startswith('L_'):
                additional_name = additional_name.replace('L_','')
        except:
            additional_name = None
    else:
        data = cfgs[name]
        try:
            additional_name = data.motor_ops.stick_to
        except:
            additional_name = None
    basic_data = data
    addit_data = cfgs[additional_name] if additional_name is not None else None
    return basic_data, addit_data

In [4]:
name = 'Pelvis'
basic_data, addit_data = get_data(cfgs, name)
basic_data, addit_data

({'mass': 5.542, 'center_of_mass': {'X': 0.0, 'Y': -0.01, 'Z': 0.91}}, None)

In [5]:
ROBOT = namedtuple('Body',['CSI', 'Masses', 'Positions', 'indices', 'jntTypes', 'axis', 'angles'])
base_position = u.coordinates(0,0,0)

In [6]:
# M_Pelvis = ROBOT(
#     CSI       = u2.geom_shape(p.GEOM_SPHERE, radius = 0.04, coeff_R = 1), 
#     Masses    = cfgs.Pelvis.mass, 
#     Positions = xyz_v2(d2m,, data = base_position),
#     indices   = 0, 
#     jntTypes  = p.JOINT_PRISMATIC, 
#     axis      = [0,0,0],
#     angles    = None)

In [8]:
def s_v2(kind,cfg):
    if kind == 'j':
        return cfg.joint_coord 
    if kind == 'm':
        return cfg.center_of_mass 
    if kind == 'd':
        return cfg

def xyz_v2(mode, BASE, STICK = None):
    if STICK is None:
        STICK = BASE
    X = s_v2(mode[1],STICK).X - s_v2(mode[0],BASE).X 
    Y = s_v2(mode[1],STICK).Y - s_v2(mode[0],BASE).Y 
    Z = s_v2(mode[1],STICK).Z - s_v2(mode[0],BASE).Z 
    return [X,Y,Z]

In [17]:
m2j = ['m','j']
j2m = ['j','m']
m2m = ['m','m']
d2m = ['d','m']
m2d = ['m','d']
xyz_v2(m2j, cfgs.AnkleF.RIGHT)

[0.0, 0.01, 0.0]

In [18]:
cfgs.TorsoF

{'mass': 2.5,
 'center_of_mass': {'X': 0.01, 'Y': 0.02, 'Z': 1.08},
 'joint_coord': {'X': 0.0, 'Y': 0.01, 'Z': 1.03},
 'motor_ops': {'motor_name': 'TorsoF',
  'stick_to': 'TorsoS',
  'axes': {'X': 0.0, 'Y': 0.0, 'Z': -1.0},
  'min_angle': -12,
  'max_angle': 40,
  'angular_speed': 75,
  'torque': 525.0}}

In [20]:
cfgs.TorsoS

{'mass': 4.105,
 'center_of_mass': {'X': 0.0, 'Y': -0.01, 'Z': 1.03},
 'joint_coord': {'X': 0.0, 'Y': 0.01, 'Z': 1.03},
 'motor_ops': {'motor_name': 'TorsoS',
  'stick_to': 'Pelvis',
  'axes': {'X': 1.0, 'Y': 0.0, 'Z': 0.0},
  'min_angle': -15,
  'max_angle': 15,
  'angular_speed': 753,
  'torque': 75.0}}

In [22]:
xyz_v2(m2j,cfgs.TorsoS,cfgs.TorsoF)
xyz_v2(j2m,cfgs.TorsoF)

[0.01, 0.01, 0.050000000000000044]

In [41]:
def dict2list(from_dict):
    return list(from_dict.values())
    
def structure(cfgs, name, index_J, index_M, VARIABLE = 0.04, j = 0.25, STATIC = [0,0,0]):
    """
    CSI : CollisionShapeIndexes
    """
    basic_data, addit_data = get_data(cfgs,name)
    motor_ops = basic_data.motor_ops
    
    J = ROBOT(
        CSI       = u2.geom_shape(p.GEOM_SPHERE, radius = VARIABLE, coeff_R = j), 
        Masses    = 0.00000001,  
        Positions = xyz_v2(m2j,addit_data, basic_data),
        indices   = index_J,
        jntTypes  = p.JOINT_REVOLUTE, 
        axis      = dict2list(motor_ops.axes), 
        angles    = [motor_ops.min_angle, motor_ops.max_angle]
    )
    M = ROBOT(
        CSI       = u2.geom_shape(p.GEOM_SPHERE, radius = VARIABLE, coeff_R = 1), 
        Masses    = basic_data.mass,  
        Positions = xyz_v2(j2m, basic_data),
        indices   = index_M, 
        jntTypes  = p.JOINT_PRISMATIC, 
        axis      = STATIC, 
        angles    = None
    )
    return J,M

In [60]:
name = 'TorsoR'
J,M = structure(cfgs, name, index_J = 0, index_M = 1)

main = cfgs[name] 
prev = cfgs[main.motor_ops.stick_to]
J.Positions == xyz_v2(m2j,prev,main), M.Positions == xyz_v2(j2m,main)

(True, True)

In [70]:
name = 'R_Foot'
R_M_Foot = ROBOT(
    CSI = u2.geom_shape(p.GEOM_BOX, height = 0.04, coeff_P = [1,3,0.25]), 
    Masses      = cfgs.Foot.RIGHT.mass, 
    Positions   = xyz_v2(m2m,cfgs.AnkleS,cfgs.Foot.RIGHT),
    indices     = 41, 
    jntTypes    = p.JOINT_PRISMATIC, 
    axis        = [0,0,0],
    angles      = None)

AttributeError: 'EasyDict' object has no attribute 'center_of_mass'

In [74]:
93* 12.32*0.85

973.896

In [69]:
R_M_Foot

Body(CSI=25, Masses=1.0, Positions=[0.0, 0.0, 0.0], indices=41, jntTypes=1, axis=[0, 0, 0], angles=None)

In [ ]:
p.disconnect()

In [8]:
import math
from time import perf_counter
from tqdm import tqdm_notebook as tqdm
from numba import njit

NameError: name 'njit' is not defined

In [ ]:
99.94230086449534